In [131]:
from tensorflow import keras
import tensorflow_datasets.public_api as tfds
import tensorflow as tf
import pathlib
import matplotlib.pyplot as plt

Создаём датасет с цветочками

In [ ]:
# (лучше загрузить свой с инета)
data_dir = tf.keras.utils.get_file(origin="https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz",
                                   fname="flower_photos",
                                   untar=True)
data_dir = pathlib.Path(data_dir)

# Константы
batch_size = 32
img_height = 180
img_width = 180

# Разбиваем датасет на тренировочную группу и группу валидации
train_data = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    image_size=(img_height, img_width),
    batch_size=batch_size,
    seed=5432,
)

validation_data = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    image_size=(img_height, img_width),
    batch_size=batch_size,
    seed=5432,
)

Модель

In [152]:
from keras.layers import (Flatten, Dense, Conv2DTranspose, Conv2D, Reshape, Input,)

# Модель
input_shape = (batch_size, img_height, img_width, 3)
latent_dim = 8

# Кодировщик
encoder_inputs = Input(shape=input_shape)
x = Conv2D(32, (3, 3), activation="relu", padding="same")(encoder_inputs)
x = Conv2D(16, (3, 3), activation="relu", padding="same")(x)
x = Conv2D(8, (3, 3), activation="relu", padding="same")(x)
x = Conv2D(4, (3, 3), activation="relu", padding="same")(x)
x = Flatten()(x)

x = Dense(latent_dim*4, activation="relu")(x)
x = Dense(latent_dim*2, activation="relu")(x)
latent_vector = Dense(latent_dim, activation="sigmoid")(x)

encoder = keras.Model(encoder_inputs, latent_vector, name="encoder")

# Декодировщик
latent_inputs = Input(shape=(batch_size, latent_dim))
x = Dense(10 * 10 * 64, activation="sigmoid")(latent_inputs)
x = Reshape((batch_size, 10, 10, 64))(x)
x = Conv2DTranspose(32, (3, 3), activation="relu", strides=2, padding="same")(x)
x = Conv2DTranspose(16, (3, 3), activation="relu", strides=2, padding="same")(x)
x = Conv2DTranspose(8, (3, 3), activation="relu", strides=2, padding="same")(x)
decoder_outputs = Conv2DTranspose(3, (3, 3), activation="sigmoid", padding="same")(x)

decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")

# Автоэнкодер
autoencoder = keras.Model(encoder_inputs, decoder(encoder(encoder_inputs)), name="autoencoder")
autoencoder.compile(
    loss="binary_crossentropy",
    optimizer="rmsprop",
)

autoencoder.summary()

ResourceExhaustedError: {{function_node __wrapped__StatelessRandomUniformV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[66355200,8] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:StatelessRandomUniformV2]